In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import numpy as np
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"
new_test = "./new_val_in/new_val_in"



# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
val_testset  = ArgoverseDataset(data_path=new_test)

In [3]:
batch_sz = 128
batch_sz_test = 1
def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = np.zeros((batch_sz, 60, 19, 2))
    out = np.zeros((batch_sz, 60, 30, 2))
    tempin = [numpy.dstack([scene['p_in']]) for scene in batch]
    tempout = [numpy.dstack([scene['p_out']]) for scene in batch]

    for l in range(4):
        for i in range(60):
            out[l][i][0][0] = tempout[l][i][0][0] - tempin[l][i][18][0]
            out[l][i][0][1] = tempout[l][i][0][1] - tempin[l][i][18][1]
            
            
            for j in range(18):
                inp[l][i][j+1][0] = tempin[l][i][j+1][0] - tempin[l][i][j][0]
                inp[l][i][j+1][1] = tempin[l][i][j+1][1] - tempin[l][i][j][1]
            for k in range(28):
                out[l][i][k+1][0] = tempout[l][i][k+1][0] - tempout[l][i][k][0]
                out[l][i][k+1][1] = tempout[l][i][k+1][1] - tempout[l][i][k][1]

#     inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
#     out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=1)

In [4]:
def my_collate1(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in']]) for scene in batch]
    scene_idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    inp = torch.FloatTensor(inp)
    return scene_idx, inp

test_loader = DataLoader(val_testset, batch_size=batch_sz_test, shuffle=False, collate_fn=my_collate1, num_workers=1)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class FCModule(nn.Module):
    
    def __init__(self):
        super(FCModule, self).__init__()
        
        self.linear1 = nn.Sequential(
                                    nn.Linear(120*19, 500), 
                                     nn.ReLU(),
                                     nn.Linear(500, 120 * 30)
                                    )

    def forward(self, x):
        x = self.linear1(x)
        
        return x

In [6]:
from tqdm import tqdm_notebook as tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = "cuda:0"
model = FCModule().to(device)
#model.load_state_dict(torch.load("usingSequential4drop_16(1)_9(1)_30"))
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

loss_ema = -1
loss_ema2 = -1

for epoch in range(10):
    for i_batch, sample_batch in enumerate (val_loader):
        inp,out = sample_batch
        inp = inp.cuda()
        out = out.cuda()
        mixed = torch.cat([inp,out],2).transpose(1,2).reshape(-1,49,120)
        y_pred = model(inp.reshape(len(inp),-1)).reshape(-1,60,30,2)
        loss = (torch.mean((y_pred-out)** 2)) ** 0.5
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if loss_ema < 0:
            loss_ema = loss
        loss_ema = loss_ema * 0.99 + loss * 0.01

        if i_batch%10 == 0:
            print('loss full', epoch, i_batch, loss_ema.item(), loss.item())
    
    torch.save(model.state_dict(),"usingSequential4data" + str(epoch))

loss full 0 0 0.047937434166669846 0.047937434166669846
loss full 0 10 0.04773908481001854 0.05600964277982712
loss full 0 20 0.04682811349630356 0.03731132671236992
loss full 0 30 0.046066027134656906 0.03582160919904709
loss full 0 40 0.04521597549319267 0.03312712907791138
loss full 0 50 0.04439274221658707 0.03654058277606964
loss full 0 60 0.04349539428949356 0.04079931974411011
loss full 0 70 0.04256511107087135 0.031063135713338852
loss full 0 80 0.04191591963171959 0.053158923983573914
loss full 0 90 0.04121364280581474 0.04206496849656105
loss full 0 100 0.04024779796600342 0.020905885845422745
loss full 0 110 0.039258044213056564 0.030628647655248642
loss full 0 120 0.038415227085351944 0.025625862181186676
loss full 0 130 0.037633832544088364 0.03248968720436096
loss full 0 140 0.03654491901397705 0.030119506642222404
loss full 0 150 0.03576910123229027 0.026570597663521767
loss full 0 160 0.035280611366033554 0.02246765047311783
loss full 0 170 0.03452226147055626 0.0298403

In [9]:
import csv
import pandas as pd
import numpy as np
import gc

gc.collect()

torch.cuda.empty_cache()

device = "cuda:0"
model = FCModule().to(device)
model.load_state_dict(torch.load("usingSequential4data4"))

filename = "output_test.csv"

def test(model, device, test_loader):
    model.eval()
    store = [[]]
    strArr = []
    strArr.append('ID')
    for i in range(60):
        strArr.append('v' + str(i + 1))


    with torch.no_grad():
        for scene_idx, data in test_loader:
            datatemp = np.zeros((1, 60, 19, 2))
            for i in range(60):
                for j in range(18):
                    datatemp[0][i][j+1][0] = data[0][i][j+1][0] - data[0][i][j][0]
                    datatemp[0][i][j+1][1] = data[0][i][j+1][1] - data[0][i][j][1]
            datatemp = torch.FloatTensor(datatemp)
            datatemp = datatemp.to(device)
            y_pred = model(datatemp.reshape(len(datatemp),-1)).reshape(-1,60,30,2)
            y_pred = y_pred[-1,0,:,0:2]
            y_pred = y_pred.cpu()
            y_pred_np = y_pred.numpy()
            y_pred_np = y_pred_np.flatten()
            y_np_temp = np.zeros(60)
            y_np_temp[0] = data[0][0][18][0] + y_pred_np[0]
            y_np_temp[1] = data[0][0][18][1] + y_pred_np[1]
            for i in range(2,len(y_pred_np)):
                y_np_temp[i] = y_np_temp[i-2] + y_pred_np[i]
            y_pred_np = y_np_temp
            y_pred_np = np.insert(y_pred_np, 0, scene_idx[0][0][0][0])
            store.append(y_pred_np)
            
    with open(filename, 'w',  newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(strArr) 
        csvwriter.writerows(store)

test(model, device, test_loader)

In [10]:
import pandas as pd

file = pd.read_csv('output_test.csv')

file["ID"] = file["ID"].astype('int32')

file.to_csv(path_or_buf="./output_test.csv", header=True, index=False)